# How to fit a model

## Data format

All model classes have a `fit` method for fitting the model to (individual or group) data, and all `fit` methods have at least 1 required argument, `data`, which should be in the form of a pandas DataFrame.

However, different model classes might require different columns in the data. You should check in the **API Reference** of each model class (or using `model.fit?`) what the required data columns are.

Below, you can see a non-learning and a learning example: 

### Non-learning example (non-hierarchical, simulated data)

In [14]:
import rlssm

In [2]:
model_ddm = rlssm.DDModel(hierarchical_levels=1)

Using cached StanModel


In [3]:
# simulate some DDM data:
from rlssm.random import simulate_ddm
data_ddm = simulate_ddm(
    n_trials=400, 
    gen_drift=.8, 
    gen_threshold=1.3, 
    gen_ndt=.23)

For the simple, non-hierarchical DDM, it is only necessary to have `rt` and `accuracy` columns:

- *rt*, response times in seconds.

- *accuracy*, 0 if the incorrect option was chosen, 1 if the correct option was chosen.

If the model is hierarchical, also include:

- *participant*, the participant number. Should be integers starting from 1 (it's also OK if `participant` is in the index).

In [4]:
data_ddm.head()

drift  rel_sp  threshold   ndt     rt  accuracy
participant trial                                                 
1           1        0.8     0.5        1.3  0.23  1.286       0.0
            2        0.8     0.5        1.3  0.23  0.374       0.0
            3        0.8     0.5        1.3  0.23  0.767       1.0
            4        0.8     0.5        1.3  0.23  1.187       1.0
            5        0.8     0.5        1.3  0.23  0.777       1.0

### Learning example (hierarchical, real data)

In [5]:
model_rl = rlssm.RLModel_2A(hierarchical_levels = 2)

Using cached StanModel


In [6]:
# import some data:
import pandas as pd
import os
par_path = os.path.abspath(os.path.join(os.getcwd(), os.pardir, os.pardir))
data_path = os.path.join(par_path, 'data/data_experiment.csv')
data_rl = pd.read_csv(data_path, index_col=0)

Since this learning model is only fit on choices, only `accuracy` (and not `rt`) is required.

Other columns that should be included are:

- *trial_block*, the number of trial in a learning session. Should be integers starting from 1.

- *f_cor*, the output from the correct option in the presented pair (the option with higher outcome on average).

- *f_inc*, the output from the incorrect option in the presented pair (the option with lower outcome on average).

- *cor_option*, the number identifying the correct option in the presented pair (the option with higher outcome on average).

- *inc_option*, the number identifying the incorrect option in the presented pair(the option with lower outcome on average).

- *block_label*, the number identifying the learning session. Should be integers starting from 1. Set to 1 in case there is only one learning session.

If the model is hierarchical, also include:

- *participant*, the participant number. Should be integers starting from 1.

If increasing_sensitivity is True, also include:

- *times_seen*, average number of times the presented options have been seen in a learning session.
  
If any of these columns are part of the index with the approriate label, they will still be recognized.

In [7]:
data_rl.head()

,participant,block_label,trial_block,f_cor,f_inc,cor_option,inc_option,times_seen,rt,accuracy
0,1,1,1,43,39,2,1,1,1.244082,0
1,1,1,2,60,50,4,3,1,1.101821,1
2,1,1,3,44,36,4,2,2,1.029923,0
3,1,1,4,55,55,4,3,2,1.368007,0
4,1,1,5,52,49,4,3,3,1.039329,1


### Additional learning parameters

While all sequential sampling models (DDM and race models) **without a learning component** only require a `data` argument, all models with a learning components (RL models, RLDDMs, and RL+race models) also require a `K` argument, which is the total number of different options in a learning block (note that this can be different from the number of options presented in each trial), and `initial_value_learning`, which is the initial Q value (before learning).

## Specify the priors

You can decide whether to use the default priors or whether you want to change the mean or SD of the prior or hyper-prior distributions.

To know what are the default priors, you can check the **API Reference** of each model class (or `model.fit?`).

## Specify the sampling parameters

The last important step is to specify the sampling parameters **(number of chains, iterations, warmups, thinning, etc.)**.

These are the arguments to the `pystan.StanModel.sampling()` function, and we simply refer to the [pystan documentation](https://pystan.readthedocs.io/) for a better overview.

## Run

In [15]:
# Run 2 chains, with 2000 samples each, 1000 of which warmup, with custom priors:
model_fit_ddm = model_ddm.fit(
    data_ddm,
    drift_priors={'mu':.5, 'sd':1},
    threshold_priors={'mu':0, 'sd':.5},
    ndt_priors={'mu':0, 'sd':.1},
    chains=2,
    iter=2000,
    warmup=1000,
    pointwise_waic=True,
    thin=1)

To run all diagnostics call pystan.check_hmc_diagnostics(fit)


Checks MCMC diagnostics:
n_eff / iter looks reasonable for all parameters
0.0 of 2000 iterations ended with a divergence (0.0%)
0 of 2000 iterations saturated the maximum tree depth of 10 (0.0%)
E-BFMI indicated no pathological behavior


In [9]:
# Run 2 chains, with 3000 samples each, 1000 of which warmup, with thinning and custom priors:
model_fit_rl = model_rl.fit(
    data_rl,
    K=4,
    initial_value_learning=27.5,
    alpha_priors={'mu_mu':-.3, 'sd_mu':.1, 'mu_sd':0, 'sd_sd':.1},
    sensitivity_priors={'mu_mu':-.1, 'sd_mu':.1, 'mu_sd':0, 'sd_sd':.1},
    chains=2,
    iter=3000,
    warmup=1000,
    print_diagnostics=False, # (not suggested, see below)
    thin=2)

To run all diagnostics call pystan.check_hmc_diagnostics(fit)


## Diagnostics

As you can see, the MCMC diagnostics are already printed by default (if you do not want this, you can set `print_diagnostics` to `False`). I refer to https://mc-stan.org/users/documentation/case-studies/divergences_and_bias.html for an excellent explanations of what these diagnostics actually mean and how to assess them.

On top of these, you can also check the convergence of the chains and the WAIC:

In [10]:
model_fit_ddm.rhat

,rhat,variable
0,0.999902,drift
1,1.001713,threshold
2,1.001130,ndt


In [11]:
model_fit_rl.rhat.describe()

,rhat
count,58.000000
mean,0.999875
std,0.000763
min,0.999067
25%,0.999291
50%,0.999658
75%,1.000173
max,1.002636


In [16]:
model_fit_ddm.waic

{'lppd': -206.31546680801432,
 'p_waic': 3.008203178578447,
 'waic': 418.6473399731855,
 'waic_se': 44.037247567155475,
 'pointwise_waic': array([ 6.86708836e+00,  9.06715564e-01,  1.18466469e+00,  4.05027111e+00,
         1.25280354e+00,  3.65799930e+00, -8.88229710e-01,  1.06884090e+00,
        -1.03569327e+00, -1.15022881e+00,  5.03448512e+00, -1.04099857e+00,
         3.82845808e+00,  1.87628948e+00,  1.83899268e+00, -1.14924792e+00,
        -9.11091144e-01,  3.15343043e+00,  6.94246394e-01, -1.02497280e+00,
        -1.81811077e-01,  2.49372371e+00,  3.56937193e+00, -6.93939374e-01,
         4.08401402e-01,  1.63447025e+00, -1.00311771e+00, -8.25998105e-01,
         1.49116505e+00,  4.08401402e-01,  1.55267238e+00,  1.35673672e+00,
        -8.32186269e-01,  4.42415932e-01,  1.19641138e+00, -1.18401195e+00,
        -6.08707350e-01, -1.19060443e+00,  2.95772754e+00, -1.17595654e+00,
        -1.22683684e+00, -9.78491963e-01, -1.00723759e-01, -9.57934728e-01,
        -8.87143691e-01,  

In [13]:
model_fit_rl.waic

{'lppd': -2632.987747986295,
 'p_waic': 52.546611787875065,
 'waic': 5371.06871954834,
 'waic_se': 94.00902560086969}

If you want to also see the point-wise WAIC, you can set `pointwise_waic` to `True`.

## Save the results

By default, the model fit results are saved in the same folder, using the `model_label` as filename. you can specify a different location using the `filename` argument.

In [17]:
model_fit_ddm.to_pickle()

Saving file as: /Users/laurafontanesi/git/rlssm/docs/notebooks/DDM.pkl


In [18]:
model_fit_rl.to_pickle()

Saving file as: /Users/laurafontanesi/git/rlssm/docs/notebooks/hierRL_2A.pkl


## Re-load previously saved results

In [19]:
model_fit_ddm = rlssm.load_model_results('/Users/laurafontanesi/git/rlssm/docs/notebooks/DDM.pkl')

In [20]:
model_fit_rl = rlssm.load_model_results('/Users/laurafontanesi/git/rlssm/docs/notebooks/hierRL_2A.pkl')

The data the model was fit on are stored in `data_info`:

In [22]:
model_fit_ddm.data_info['data']

,index,participant,trial,drift,rel_sp,threshold,ndt,rt,accuracy,accuracy_neg,accuracy_flipped
0,0,1,1,0.8,0.5,1.3,0.23,1.286,0.0,-1,1.0
1,1,1,2,0.8,0.5,1.3,0.23,0.374,0.0,-1,1.0
2,2,1,3,0.8,0.5,1.3,0.23,0.767,1.0,1,-0.0
3,3,1,4,0.8,0.5,1.3,0.23,1.187,1.0,1,-0.0
4,4,1,5,0.8,0.5,1.3,0.23,0.777,1.0,1,-0.0
...,...,...,...,...,...,...,...,...,...,...,...
395,395,1,396,0.8,0.5,1.3,0.23,0.610,1.0,1,-0.0
396,396,1,397,0.8,0.5,1.3,0.23,0.699,1.0,1,-0.0
397,397,1,398,0.8,0.5,1.3,0.23,1.205,1.0,1,-0.0
398,398,1,399,0.8,0.5,1.3,0.23,0.388,0.0,-1,1.0


And different parameter information are stored in `parameter_info`:

In [25]:
model_fit_rl.parameters_info

{'hierarchical_levels': 2,
 'n_parameters_individual': 2,
 'n_parameters_trial': 0,
 'n_posterior_samples': 2000,
 'n_parameters_group': 4,
 'n_parameters_hierarchical': 58,
 'parameters_names': ['mu_alpha',
  'mu_sensitivity',
  'sd_alpha',
  'sd_sensitivity',
  'z_alpha[1]',
  'z_alpha[2]',
  'z_alpha[3]',
  'z_alpha[4]',
  'z_alpha[5]',
  'z_alpha[6]',
  'z_alpha[7]',
  'z_alpha[8]',
  'z_alpha[9]',
  'z_alpha[10]',
  'z_alpha[11]',
  'z_alpha[12]',
  'z_alpha[13]',
  'z_alpha[14]',
  'z_alpha[15]',
  'z_alpha[16]',
  'z_alpha[17]',
  'z_alpha[18]',
  'z_alpha[19]',
  'z_alpha[20]',
  'z_alpha[21]',
  'z_alpha[22]',
  'z_alpha[23]',
  'z_alpha[24]',
  'z_alpha[25]',
  'z_alpha[26]',
  'z_alpha[27]',
  'z_sensitivity[1]',
  'z_sensitivity[2]',
  'z_sensitivity[3]',
  'z_sensitivity[4]',
  'z_sensitivity[5]',
  'z_sensitivity[6]',
  'z_sensitivity[7]',
  'z_sensitivity[8]',
  'z_sensitivity[9]',
  'z_sensitivity[10]',
  'z_sensitivity[11]',
  'z_sensitivity[12]',
  'z_sensitivity[13]'